<h2 style="color:blue;"> <strong>RAG but better - MultiQueryRetriever</strong> 🚀</h2>

In [21]:
import os
import psycopg2
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
from langchain_community.document_loaders.pdf import PyPDFium2Loader

In [8]:
async def load_all_pdfs_from_folder(folder_path: str):
    folder = Path(folder_path)
    
    if not folder.is_dir():
        raise FileNotFoundError(f"The folder {folder_path} does not exist.")

    pdf_files = list(folder.glob("*.pdf"))
    
    if not pdf_files:
        raise FileNotFoundError("No PDF files found in the specified folder.")

    all_pages = []

    for pdf_file in pdf_files:
        loader = PyPDFium2Loader(str(pdf_file))
        async for page in loader.alazy_load():
            all_pages.append(page)

    return all_pages

In [9]:
folder_path = "data"
pages = await load_all_pdfs_from_folder(folder_path)

/home/martyna/.cache/pypoetry/virtualenvs/rag-multiqueryretrieval-8FmIK_BO-py3.12/lib/python3.12/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [12]:
print(type(pages[0]), len(pages))

<class 'langchain_core.documents.base.Document'> 99


In [16]:
{page.metadata.get('source') for page in pages}

{'data/AI Meets the Classroom: When Does ChatGPT Harm Learning?.pdf',
 'data/Generative AI Can Harm Learning.pdf'}

### **Connect to PostgreSQL**

In [29]:
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database!")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit()

cur = conn.cursor()

try:
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
    """)
    tables = cur.fetchall()
    
    print("\nTables in the database:")
    for table in tables:
        print(table[0])
except Exception as e:
    print(f"Error fetching table names: {e}")


cur.close()
conn.close()

Connected to the database!

Tables in the database:
embeddings_256
